### Data Cleaning and Merging Dataframes

In [1]:
import pickle
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import seaborn as sns
import datetime

In [3]:
# Import all the dataframes
mojog_df = pd.read_pickle('mojog_df.pkl')
#unemp_df = pd.read_pickle("data/unemp_df.pkl")
#youtube_df = pd.read_pickle('data/youtube_df.pkl')
omdb_df = pd.read_pickle('omdb_df.pkl')
director_df = pd.read_pickle('director_df.pkl')
actor_df = pd.read_pickle('actor_df.pkl')

In [4]:
#clean mojo df
mojog_df.opening = mojog_df.opening.replace("\$","", regex = True).replace(",","", regex = True).str.strip()
mojog_df.opening = pd.to_numeric(mojog_df.opening)
mojog_df.tot_gross = mojog_df.tot_gross.replace("\$","", regex = True).replace(",","", regex = True).str.strip()
mojog_df.tot_gross = pd.to_numeric(mojog_df.tot_gross)
mojog_df.theaters = mojog_df.theaters.replace(",","", regex = True).str.strip()
mojog_df.theaters = pd.to_numeric(mojog_df.theaters)
mojog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 599
Data columns (total 9 columns):
movie_name    600 non-null object
movie         600 non-null object
opening       600 non-null int64
date_time     600 non-null datetime64[ns]
theaters      600 non-null int64
tot_gross     600 non-null int64
actors        578 non-null object
directors     549 non-null object
gtrend        600 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 46.9+ KB


In [5]:
mojog_df.head()

,movie_name,movie,opening,date_time,theaters,tot_gross,actors,directors,gtrend
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,2013-11-22,4163,424668047,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],0.564
1,Iron Man 3,Iron Man 3,174144585,2013-05-03,4253,409013994,"[Robert Downey, Jr., Gwyneth Paltrow, Don Chea...",[Shane Black],0.707
2,Frozen,Frozen,243390,2013-11-22,1,400738009,"[Kristen Bell, Josh Gad, Alan Tudyk]","[Chris Buck, Jennifer Lee]",0.123
3,Despicable Me 2,Despicable Me 2,83517315,2013-07-03,3997,368061265,"[Steve Carell, Kristen Wiig, Benjamin Bratt, R...","[Pierre Coffin, Chris Renaud]",0.656
4,Man of Steel,Man of Steel,116619362,2013-06-14,4207,291045518,"[Henry Cavill, Amy Adams, Michael Shannon, Kev...",[Zack Snyder],0.607


In [4]:
# Clean youtube df 
# select youtube df relevant data
youtube_df = youtube_df[["movie_name", "viewCount", "commentCount", "dislikeCount", "likeCount"]]

#replace nulls with 0
youtube_df = youtube_df.fillna(0)
# convert numeric str values to int
youtube_df.viewCount = pd.to_numeric(youtube_df.viewCount)
youtube_df.dislikeCount = pd.to_numeric(youtube_df.dislikeCount)
youtube_df.commentCount = pd.to_numeric(youtube_df.commentCount)
youtube_df.likeCount = pd.to_numeric(youtube_df.likeCount)
youtube_df = youtube_df.rename(columns ={"viewCount":"Yviews", "commentCount": "Ycomments", 
                                         "dislikeCount": "Ydislikes", "likeCount":"Ylikes"})
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
movie_name    500 non-null object
Yviews        500 non-null int64
Ycomments     500 non-null int64
Ydislikes     500 non-null int64
Ylikes        500 non-null int64
dtypes: int64(4), object(1)
memory usage: 19.6+ KB


In [8]:
# Clean omdb data
#omdb_df.imdb = pd.to_numeric(omdb_df.imdb.replace("/10","", regex = True).str.strip())
omdb_df.metacritic = pd.to_numeric(omdb_df.metacritic.replace("/100","", regex = True).str.strip())
omdb_df.rotten_tomatoes = pd.to_numeric(omdb_df.rotten_tomatoes.replace("%","", regex = True).str.strip())
omdb_df.runtime = pd.to_numeric(omdb_df.runtime.replace("min","", regex = True).str.strip())
omdb_df.year = pd.to_numeric(omdb_df.year)
# Remove 11 movies that have been wrongly identified on omdb (wrong year)
year = [2013,2014,2015,2016,2017,2018]
omdb_df = omdb_df[omdb_df.year.isin(year)]
omdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 0 to 573
Data columns (total 7 columns):
imdb               559 non-null float64
metacritic         559 non-null int64
movie              559 non-null object
rated              559 non-null object
rotten_tomatoes    559 non-null int64
runtime            559 non-null int64
year               559 non-null int64
dtypes: float64(1), int64(4), object(2)
memory usage: 34.9+ KB


In [9]:
omdb_df.head()

,imdb,metacritic,movie,rated,rotten_tomatoes,runtime,year
0,7.5,76,The Hunger Games: Catching Fire,PG-13,89,146,2013
1,7.2,62,Iron Man 3,PG-13,79,130,2013
2,7.5,74,Frozen,PG,90,102,2013
3,7.4,62,Despicable Me 2,PG,74,98,2013
4,7.1,55,Man of Steel,PG-13,56,143,2013


In [13]:
# Clean directors data
# Convert str to numeric
#director_df.dir_agross = pd.to_numeric(director_df.dir_agross.replace("\$","", regex = True).str.strip())
#director_df.dir_gross = pd.to_numeric(director_df.dir_gross.replace("\$","", regex = True)
#                                      .replace(",","", regex = True).str.strip())
director_df.dir_nmovies = pd.to_numeric(director_df.dir_nmovies)
director_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
dir_agross     500 non-null float64
dir_gross      500 non-null float64
dir_nmovies    500 non-null int64
director       500 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 15.7+ KB


In [15]:
director_df.head()

,dir_agross,dir_gross,dir_nmovies,director
0,141.6,4672.2,33,Steven Spielberg
1,179.0,2326.5,13,Michael Bay
2,165.6,2152.7,13,Peter Jackson
3,84.4,2109.8,25,Ron Howard
4,110.8,2104.3,19,Robert Zemeckis


In [16]:
# Clean directors data
# Convert str to numeric
actor_df.act_agross = pd.to_numeric(actor_df.act_agross.replace("\$", "", regex = True)
                                    .replace(",", "", regex = True).str.strip())
actor_df.act_gross = pd.to_numeric(actor_df.act_gross.replace("\$", "", regex = True)
                                      .replace(",", "", regex = True).str.strip())
actor_df.act_nmovies = pd.to_numeric(actor_df.act_nmovies)
actor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
act_agross     500 non-null float64
act_gross      500 non-null float64
act_nmovies    500 non-null int64
actor          500 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 15.7+ KB


In [17]:
actor_df.head()

,act_agross,act_gross,act_nmovies,actor
0,86.9,6693.5,77,Samuel L. Jackson
1,100.4,5620.8,56,Robert Downey Jr.
2,117.4,5046.2,43,Scarlett Johansson
3,118.2,4963.8,42,Harrison Ford
4,96.0,4606.8,48,Tom Hanks


In [10]:
mojog_df.head()

,movie_name,movie,opening,date_time,theaters,tot_gross,actors,directors,gtrend
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,2013-11-22,4163,424668047,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],0.456
1,Iron Man 3,Iron Man 3,174144585,2013-05-03,4253,409013994,"[Robert Downey, Jr., Gwyneth Paltrow, Don Chea...",[Shane Black],0.792
2,Frozen,Frozen,243390,2013-11-22,1,400738009,"[Kristen Bell, Josh Gad, Alan Tudyk]","[Chris Buck, Jennifer Lee]",0.124
3,Despicable Me 2,Despicable Me 2,83517315,2013-07-03,3997,368061265,"[Steve Carell, Kristen Wiig, Benjamin Bratt, R...","[Pierre Coffin, Chris Renaud]",0.706
4,Man of Steel,Man of Steel,116619362,2013-06-14,4207,291045518,"[Henry Cavill, Amy Adams, Michael Shannon, Kev...",[Zack Snyder],0.713


In [11]:
# Create a new column unemp_date, by dropping the datetime day to 1
# This is done in order to merge the unemployment monthly data to the movies release dates
#datetime.date(2008,1,1).replace(day=5)
mojog_df["unemp_date"] = mojog_df.date_time.apply(lambda x: x.replace(day=1))
mojog_df = pd.merge(mojog_df,unemp_df,on="unemp_date",how="inner")
mojog_df = mojog_df.drop(["unemp_date"],1)
mojog_df.head()

,movie_name,movie,opening,date_time,theaters,tot_gross,actors,directors,gtrend,unemp_rate
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,2013-11-22,4163,424668047,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],0.456,6.9
1,Frozen,Frozen,243390,2013-11-22,1,400738009,"[Kristen Bell, Josh Gad, Alan Tudyk]","[Chris Buck, Jennifer Lee]",0.124,6.9
2,Thor: The Dark World,Thor: The Dark World,85737841,2013-11-08,3841,206362140,"[Chris Hemsworth, Natalie Portman, Tom Hiddles...",[Alan Taylor],0.474,6.9
3,The Best Man Holiday,The Best Man Holiday,30107555,2013-11-15,2024,70525195,"[Morris Chestnut, Regina Hall, Terrence Howard...",[Malcolm D. Lee],0.220,6.9
4,Last Vegas,Last Vegas,16334566,2013-11-01,3065,63914167,"[Michael Douglas, Robert DeNiro, Morgan Freema...",[Jon Turteltaub],0.187,6.9


# Merge the dataframes

In [21]:
merged2 = pd.DataFrame.merge(mojog_df, omdb_df,on='movie', how = 'inner')
merged2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 0 to 558
Data columns (total 15 columns):
movie_name         559 non-null object
movie              559 non-null object
opening            559 non-null int64
date_time          559 non-null datetime64[ns]
theaters           559 non-null int64
tot_gross          559 non-null int64
actors             538 non-null object
directors          510 non-null object
gtrend             559 non-null float64
imdb               559 non-null float64
metacritic         559 non-null int64
rated              559 non-null object
rotten_tomatoes    559 non-null int64
runtime            559 non-null int64
year               559 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(7), object(5)
memory usage: 69.9+ KB


In [22]:
merged2 = merged2.fillna(0)
merged2.head()

,movie_name,movie,opening,date_time,theaters,tot_gross,actors,directors,gtrend,imdb,metacritic,rated,rotten_tomatoes,runtime,year
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,2013-11-22,4163,424668047,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],0.564,7.5,76,PG-13,89,146,2013
1,Iron Man 3,Iron Man 3,174144585,2013-05-03,4253,409013994,"[Robert Downey, Jr., Gwyneth Paltrow, Don Chea...",[Shane Black],0.707,7.2,62,PG-13,79,130,2013
2,Frozen,Frozen,243390,2013-11-22,1,400738009,"[Kristen Bell, Josh Gad, Alan Tudyk]","[Chris Buck, Jennifer Lee]",0.123,7.5,74,PG,90,102,2013
3,Despicable Me 2,Despicable Me 2,83517315,2013-07-03,3997,368061265,"[Steve Carell, Kristen Wiig, Benjamin Bratt, R...","[Pierre Coffin, Chris Renaud]",0.656,7.4,62,PG,74,98,2013
4,Man of Steel,Man of Steel,116619362,2013-06-14,4207,291045518,"[Henry Cavill, Amy Adams, Michael Shannon, Kev...",[Zack Snyder],0.607,7.1,55,PG-13,56,143,2013


# Generate a metric for actor and directors

#### actors
I took the max total gross or total # movies featured for the actor/director for each given movie.

In [23]:
import numpy as np

act_list = []
for i in range(len(merged2)):
    #for each of the actors in a given movie, sum up their total gross & movies
    gross_list = []
    nmovies_list = []
    if (merged2.actors[i] == 0) or (merged2.actors[i] == []):
        act_gross, act_nmovies = 0, 0
    else:
        actor_count = len(merged2.actors[i])
        for actoriter in merged2.actors[i]:
            if actor_df[actor_df.actor == actoriter].empty:
                actor_count -= 1
            #Subtract the movie total gross (movie i want to predict) from actor total gross
            else:
                gross_list.append(actor_df[actor_df.actor == actoriter].act_gross.iloc[0] - \
                merged2.tot_gross[i]/(1e6))
                nmovies_list.append(actor_df[actor_df.actor == actoriter].act_nmovies.iloc[0])
    #take the sum and average over number of actors featured
    if (actor_count <= 0) or (merged2.actors[i] == []) or (merged2.actors[i] == 0):
        act_gross, act_nmovies = 0, 0
    else:
        act_gross = round(max(gross_list),1)
        act_nmovies = round(max(nmovies_list),1)
    act_dict = {'movie': merged2.movie[i], 'act_gross':act_gross, 
                'act_nmovies': act_nmovies}
    act_list.append(act_dict)
actor_metric = pd.DataFrame(act_list)

#### directors

In [24]:
import numpy as np

dir_list = []
for i in range(len(merged2)):
    #for each of the actors in a given movie, sum up their total gross & movies
    gross_list = []
    nmovies_list = []
    if (merged2.directors[i] == 0) or (merged2.directors[i] == []):
        dir_gross, dir_nmovies = 0, 0
    else:
        director_count = len(merged2.directors[i])
        for directoriter in merged2.directors[i]:
            if director_df[director_df.director == directoriter].empty:
                director_count -= 1
            #Subtract the movie total gross (movie i want to predict) from actor total gross
            else:
                gross_list.append(director_df[director_df.director == directoriter].
                                  dir_gross.iloc[0] - \
                merged2.tot_gross[i]/(1e6))
                nmovies_list.append(director_df[director_df.director == directoriter]
                                    .dir_nmovies.iloc[0])
    #take the sum and average over number of actors featured
    if (director_count <= 0) or (merged2.directors[i] == []) or (merged2.directors[i] == 0):
        dir_gross, dir_nmovies = 0, 0
    else:
        dir_gross = round(max(gross_list),1)
        dir_nmovies = round(max(nmovies_list),1)
    dir_dict = {'movie': merged2.movie[i], 'dir_gross':dir_gross, 
                'dir_nmovies': dir_nmovies}
    dir_list.append(dir_dict)
director_metric = pd.DataFrame(dir_list)

In [27]:
actor_metric.head()

,act_gross,act_nmovies,movie
0,3353.5,59,The Hunger Games: Catching Fire
1,3815.7,53,Iron Man 3
2,2098.8,19,Frozen
3,2686.0,33,Despicable Me 2
4,2441.1,43,Man of Steel


In [28]:
director_metric.head()

,dir_gross,dir_nmovies,movie
0,1056.8,7,The Hunger Games: Catching Fire
1,91.5,4,Iron Man 3
2,230.0,3,Frozen
3,852.1,4,Despicable Me 2
4,1028.6,8,Man of Steel


# Merge the director & actor metrics to df

In [25]:
merged2 = pd.merge(merged2, actor_metric, on="movie", how="inner")
merged2 = pd.merge(merged2, director_metric, on="movie", how="inner")
#merged2 = merged2.drop(["actors","directors"],1)
merged2.head()

,movie_name,movie,opening,date_time,theaters,tot_gross,actors,directors,gtrend,imdb,metacritic,rated,rotten_tomatoes,runtime,year,act_gross,act_nmovies,dir_gross,dir_nmovies
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,158074286,2013-11-22,4163,424668047,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",[Francis Lawrence],0.564,7.5,76,PG-13,89,146,2013,3353.5,59,1056.8,7
1,Iron Man 3,Iron Man 3,174144585,2013-05-03,4253,409013994,"[Robert Downey, Jr., Gwyneth Paltrow, Don Chea...",[Shane Black],0.707,7.2,62,PG-13,79,130,2013,3815.7,53,91.5,4
2,Frozen,Frozen,243390,2013-11-22,1,400738009,"[Kristen Bell, Josh Gad, Alan Tudyk]","[Chris Buck, Jennifer Lee]",0.123,7.5,74,PG,90,102,2013,2098.8,19,230.0,3
3,Despicable Me 2,Despicable Me 2,83517315,2013-07-03,3997,368061265,"[Steve Carell, Kristen Wiig, Benjamin Bratt, R...","[Pierre Coffin, Chris Renaud]",0.656,7.4,62,PG,74,98,2013,2686.0,33,852.1,4
4,Man of Steel,Man of Steel,116619362,2013-06-14,4207,291045518,"[Henry Cavill, Amy Adams, Michael Shannon, Kev...",[Zack Snyder],0.607,7.1,55,PG-13,56,143,2013,2441.1,43,1028.6,8


### Save the merged dataframe

In [29]:
merged2.to_pickle("merged_df.pkl")

In [30]:
len(merged2[(merged2.dir_nmovies == 0)])# | (merged2.act_gross == 0)])

149